In [ ]:
import os, json, cv2, yaml, torch
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 경로 설정
BASE_DIR = Path.cwd().parent.parent
IMG_DIR = BASE_DIR / "data/images"
JSON_DIR = BASE_DIR / "data/json_labels"
DATASET = BASE_DIR / "processed/preprocessed_data/yolov5"
RESULT_DIR = BASE_DIR / "processed/results_comparison/yolov5"

for d in [DATASET, RESULT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# 전처리 클래스
class Preprocessor:
    def __init__(self, size=640):
        self.size = size
        self.classes = []
        self.class_to_idx = {}

    def _resize(self, img_path, save_path):
        img = cv2.imread(str(img_path))
        if img is None:
            return None
        h, w = img.shape[:2]
        scale = min(self.size / w, self.size / h)
        new = cv2.resize(img, (int(w * scale), int(h * scale)))
        pad_h = (self.size - new.shape[0]) // 2
        pad_w = (self.size - new.shape[1]) // 2
        padded = cv2.copyMakeBorder(
            new, pad_h, self.size - new.shape[0] - pad_h,
            pad_w, self.size - new.shape[1] - pad_w,
            cv2.BORDER_CONSTANT, value=(114, 114, 114)
        )
        save_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path), padded)
        return scale, pad_h, pad_w

    def _bbox(self, b, s, pt, pl):
        x1, x2 = b['xmin'] * s + pl, b['xmax'] * s + pl
        y1, y2 = b['ymin'] * s + pt, b['ymax'] * s + pt
        return [
            (x1 + x2) / 2 / self.size,
            (y1 + y2) / 2 / self.size,
            (x2 - x1) / self.size,
            (y2 - y1) / self.size
        ]

    def _class_id(self, c1, c3):
        name = f"{c1}_{c3}"
        if name not in self.classes:
            self.class_to_idx[name] = len(self.classes)
            self.classes.append(name)
        return self.class_to_idx[name]

    def run(self):
        jsons = list(JSON_DIR.glob("*.json"))
        if not jsons:
            return []

        for split in ['train', 'val', 'test']:
            (DATASET / 'labels' / split).mkdir(parents=True, exist_ok=True)

        train, temp = train_test_split(jsons, train_size=0.8, random_state=42)
        val, test = train_test_split(temp, train_size=0.5, random_state=42)

        for split, files in zip(['train', 'val', 'test'], [train, val, test]):
            for j in tqdm(files, desc=f"{split}"):
                with open(j, 'r', encoding='utf-8') as f:
                    d = json.load(f)
                
                # 이미지 찾기
                img = None
                for ext in ['.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG']:
                    img_path = IMG_DIR / f"{j.stem}{ext}"
                    if img_path.exists():
                        img = img_path
                        break
                
                if not img:
                    continue

                save_img = DATASET / 'images' / split / f"{j.stem}.png"
                r = self._resize(img, save_img)
                if not r:
                    continue

                s, pt, pl = r
                x, y, w, h = self._bbox(d['bndbox'], s, pt, pl)
                cid = self._class_id(d['cate1'], d['cate3'])

                with open(DATASET / 'labels' / split / f"{j.stem}.txt", 'w') as f:
                    f.write(f"{cid} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

        # data.yaml 저장
        with open(DATASET / 'data.yaml', 'w', encoding='utf-8') as f:
            yaml.dump({
                'path': str(DATASET),
                'train': 'images/train',
                'val': 'images/val',
                'test': 'images/test',
                'nc': len(self.classes),
                'names': self.classes
            }, f, allow_unicode=True)
        
        return self.classes


# 학습 함수
def train_yolo(data_yaml, model_size="s", epochs=50, batch=16, device="0"):
    model = YOLO(f"yolov5{model_size}.pt")
    results = model.train(
        data=str(data_yaml),
        epochs=epochs,
        imgsz=640,
        batch=batch,
        name="freshness_yolov5",
        device=device,
        patience=15,
        workers=0,
        project=str(BASE_DIR / "runs")
    )
    return model


# 평가 클래스
class Evaluator:
    def __init__(self, model_path, device):
        self.model = YOLO(model_path)
        self.device = device
        self.names = self.model.names

    def evaluate(self, data_yaml):
        print("\n검증 세트 평가 중...")
        metrics = self.model.val(data=str(data_yaml), device=self.device)
        print(f"\nmAP50: {metrics.box.map50:.3f}")
        print(f"mAP50-95: {metrics.box.map:.3f}")
        print(f"Precision: {metrics.box.mp:.3f}")
        print(f"Recall: {metrics.box.mr:.3f}")
        return metrics

    def visualize(self, img_path, save_path=None):
        r = self.model.predict(source=img_path, conf=0.25, imgsz=640, device=self.device)[0]
        img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
        
        fig, ax = plt.subplots(1, figsize=(10, 10))
        ax.imshow(img)
        
        for b in r.boxes:
            x1, y1, x2, y2 = b.xyxy[0].cpu().numpy()
            conf = b.conf[0].cpu().numpy()
            cls = int(b.cls[0])
            name = self.names[cls]
            
            color = "green" if "특상" in name else ("red" if "상" in name else "blue")
            
            ax.add_patch(patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=3, edgecolor=color, facecolor="none"
            ))
            ax.text(x1, y1 - 10, f"{name} {conf:.2f}",
                   bbox=dict(facecolor=color, alpha=0.7),
                   color="white", fontsize=10)
        
        ax.axis("off")
        if save_path:
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, bbox_inches="tight", dpi=150)
        plt.show()


# 메인 실행
def main():
    print("\nYOLOv5 신선도 분류 파이프라인 시작\n")

    # 1. 전처리
    print("1. 데이터 전처리 중...")
    pre = Preprocessor()
    classes = pre.run()
    if not classes:
        print("데이터셋이 비어 있습니다.")
        return
    print(f"클래스: {classes}\n")

    # 2. 학습
    data_yaml = DATASET / "data.yaml"
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"2. 학습 시작 (Device: {device})")
    
    model = train_yolo(data_yaml, model_size="s", epochs=50, batch=16, device=device)

    # 3. 평가
    print("\n3. 모델 평가")
    best_w = BASE_DIR / "runs/freshness_yolov5/weights/best.pt"
    
    if not best_w.exists():
        print(f"모델 가중치를 찾을 수 없습니다: {best_w}")
        return
    
    evaluator = Evaluator(best_w, device)
    evaluator.evaluate(data_yaml)

    # 4. 시각화
    print("\n4. 결과 시각화")
    test_imgs = list((DATASET / "images/test").glob("*.png"))[:3]
    
    if test_imgs:
        for img in test_imgs:
            evaluator.visualize(img, RESULT_DIR / f"{img.stem}_result.png")
        print(f"시각화 결과: {RESULT_DIR}")
    else:
        print("테스트 이미지가 없습니다.")

    print("\n모든 단계 완료!")


if __name__ == "__main__":
    main()


YOLOv5 신선도 분류 파이프라인 시작

1. 데이터 전처리 중...


test: 100%|██████████| 7/7 [00:00<00:00, 24.88it/s]


클래스: ['감_보통', '배_보통', '감_상', '감_특', '사과_상', '사과_보통', '사과_특']

2. 학습 시작 (Device: cuda:0)
PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

New https://pypi.org/project/ultralytics/8.3.226 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.225  Python-3.9.25 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\miniproject\data\dataset_yolo\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist

c:\Users\smart\miniconda3\envs\tf01\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 39.94.8 MB/s, size: 466.0 KB)
val: Scanning C:\miniproject\data정리\dataset_yolo\labels\val.cache... 7 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 7/7  0.0s
Plotting labels to C:\miniproject\data\runs\freshness_yolov52\labels.jpg... 


...

C:\Users\smart\AppData\Local\Temp\ipykernel_28348\970428613.py:173: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  plt.savefig(save_path, bbox_inches="tight", dpi=150)


<Figure size 1000x1000 with 1 Axes>


image 1/1 c:\miniproject\data\dataset_yolo\images\test\apple_fuji_M_26-61.png: 640x640 1 _, 1 _, 97.8ms
Speed: 4.3ms preprocess, 97.8ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1000x1000 with 1 Axes>

시각화 결과: c:\miniproject\data정리\results

모든 단계 완료!
